# GWAS via REGENIE for PP

This uses dsub (as opposed to WDL + Cromwell) to submit bash scripts corresponding to REGENIE for GWAS.

Template for code: https://workbench.researchallofus.org/workspaces/aou-rw-5981f9dc/aouldlgwasregeniedsubctv6duplicate/analysis/preview/4.0_regenie_dsub_HP_TM.ipynb 
i.e the GWAS for LDL-C done by Bick et al, 2024

Modified for my appplications.

In [4]:
## Python Package Import
import sys
import os 
import numpy as np
import pandas as pd
from datetime import datetime

##Ensuring dsub is up to date
!pip3 install --upgrade dsub

In [5]:
#Defining environment variables
# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env JOB_ID={LINE_COUNT_JOB_ID}
## Defining necessary pathways
my_bucket = os.environ['WORKSPACE_BUCKET']
## Setting for running dsub jobs
pd.set_option('display.max_colwidth', 0)

USER_NAME = os.getenv('OWNER_EMAIL').split('@')[0].replace('.','-')

# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env USER_NAME={USER_NAME}

env: JOB_ID={LINE_COUNT_JOB_ID}
env: USER_NAME=jon126


## PLINK QC Step

This runs the code and submits the job for each chromosome to run the preparatory QC step which is phenotype-agnostic.

In [6]:
## MODIFY FOR FULL DATA RUN 
JOB_NAME='1_regenie_plinkprep'

# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env JOB_NAME={JOB_NAME}

env: JOB_NAME=1_regenie_plinkprep


In [7]:
## Analysis Results Folder 
line_count_results_folder = os.path.join(
    os.getenv('WORKSPACE_BUCKET'),
    'dsub',
    'results',
    JOB_NAME,
    USER_NAME,
    datetime.now().strftime('%Y%m%d'))

line_count_results_folder

'gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/results/1_regenie_plinkprep/jon126/20241001'

In [8]:
## Where the output files will go
output_files = os.path.join(line_count_results_folder, "results")
print(output_files)

OUTPUT_FILES = output_files

# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env OUTPUT_FILES={OUTPUT_FILES}

gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/results/1_regenie_plinkprep/jon126/20241001/results
env: OUTPUT_FILES=gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/results/1_regenie_plinkprep/jon126/20241001/results


In [24]:
#This is the plink preparatory in REGENIE 
filename='1_regenie_plinkprep.sh'

script = '''

set -o pipefail 
set -o errexit

plink \
    --bed "${bedfile}" \
    --bim "${bimfile}" \
    --fam "${famfile}" \
    --threads 3 \ 
    --geno 0.01 \
    --hwe 1e-15 \
    --indep-pairwise 1000 100 0.8 \
    --write-snplist \
    --out qc_ldpruned_snps_chr"${chr}"

export output_snplist="qc_ldpruned_snps_chr${chr}.snplist" 
export output_ldprune_in="qc_ldpruned_snps_chr${chr}.prune.in" 
export output_ldprune_out="qc_ldpruned_snps_chr${chr}.prune.out"

mv ${output_snplist} ${output_ldprune_in} ${output_ldprune_out} -t ${OUTPUT_PATH}
'''

with open(filename,'w') as fp:
    fp.write(script)


In [25]:
!gsutil cp ./1_regenie_plinkprep.sh {my_bucket}/dsub/scripts/ 

Copying file://./1_regenie_plinkprep.sh [Content-Type=text/x-sh]...
/ [1 files][  527.0 B/  527.0 B]                                                
Operation completed over 1 objects/527.0 B.                                      


In [10]:
#Check the file is there
!gsutil ls {my_bucket}/dsub/scripts/*.sh

gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/scripts/1_regenie_plinkprep.sh
gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/scripts/2_regenie_aous_pp.sh


In [9]:
# !gsutil -u $GOOGLE_PROJECT ls -lh gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/

423.72 GiB  2023-09-26T17:40:39Z  gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr1.bed
356.34 MiB  2023-09-26T13:42:19Z  gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr1.bim
  4.45 MiB  2023-09-26T13:41:54Z  gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr1.fam
272.02 GiB  2023-09-26T16:05:59Z  gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr10.bed
238.88 MiB  2023-09-26T13:42:14Z  gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr10.bim
  4.45 MiB  2023-09-26T13:41:52Z  gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr10.fam
253.58 GiB  2023-09-26T15:59:01Z  gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/pl

In [17]:
%%bash --out LINE_COUNT_JOB_ID
#This submits the job btw for each chromosome!

# Get a shorter username to leave more characters for the job name.
DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

# For AoU RWB projects network name is "network".
AOU_NETWORK=network
AOU_SUBNETWORK=subnetwork

MACHINE_TYPE="n2-standard-4"
BASH_SCRIPT="gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/scripts/1_regenie_plinkprep.sh" #From above command

# Python is 'right side limited' wherein the last value is not included
# To run the regression across all chromosomes, set lower to 1 and upper to 23
# To run across one chromosome, set lower to the chomosome-of-interest and upper to the following

# LOWER=1
# UPPER=23

#Test on 1 chromosome first
LOWER=15
UPPER=16
for ((chromo=$LOWER;chromo<$UPPER;chromo+=1))
do
    dsub \
    --provider google-cls-v2 \
    --user-project "${GOOGLE_PROJECT}" \
    --project "${GOOGLE_PROJECT}" \
    --image "us.gcr.io/broad-dsp-gcr-public/terra-jupyter-aou:2.2.14" \
    --network "${AOU_NETWORK}" \
    --subnetwork "${AOU_SUBNETWORK}" \
    --service-account "$(gcloud config get-value account)" \
    --user "${DSUB_USER_NAME}" \
    --regions us-central1 \
    --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/$(date +'%Y%m%d/%H%M%S')/{job-id}-{task-id}-{task-attempt}.log" \
    "$@" \
    --preemptible \
    --disk-size 1000 \
    --boot-disk-size 500 \
    --machine-type ${MACHINE_TYPE} \
    --name "${JOB_NAME}" \
    --script "${BASH_SCRIPT}" \
    --env GOOGLE_PROJECT=${GOOGLE_PROJECT} \
    --input bedfile="gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr${chromo}.bed" \
    --input bimfile="gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr${chromo}.bim" \
    --input famfile="gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr${chromo}.fam" \
    --env chr=${chromo} \
    --output-recursive OUTPUT_PATH="${OUTPUT_FILES}/${chromo}"
done

Job properties:
  job-id: 1-regenie---jon126--241001-113153-36
  job-name: 1-regenie-plinkprep
  user-id: jon126
Provider internal-id (operation): projects/282373700333/locations/us-central1/operations/1771003859732048182
Launched job-id: 1-regenie---jon126--241001-113153-36
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-4e1b6fe8 --location us-central1 --jobs '1-regenie---jon126--241001-113153-36' --users 'jon126' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-4e1b6fe8 --location us-central1 --jobs '1-regenie---jon126--241001-113153-36' --users 'jon126'


In [18]:
# Check the status of your job submissions

!dstat \
    --provider google-cls-v2 \
    --project terra-vpc-sc-4e1b6fe8 \
    --location us-central1 \
    --jobs '*' \
    --users 'jon126' \
    --status '*'

Job Name         Status                                      Last Update
---------------  ------------------------------------------  --------------
1-regenie-pl...  Started running "user-command"              10-01 12:29:26
1-regenie-pl...  Stopped running "user-command": exit st...  09-30 18:31:52
1-regenie-pl...  Stopped running "user-command": exit st...  09-30 19:05:05
1-regenie-pl...  worker was terminated                       09-30 16:41:42
1-regenie-pl...  Stopped running "user-command": exit st...  09-30 20:37:09
1-regenie-pl...  Stopped running "user-command": exit st...  09-30 21:11:09
1-regenie-pl...  worker was terminated                       09-30 20:35:34
1-regenie-pl...  worker was terminated                       09-30 17:24:32
1-regenie-pl...  Stopped running "user-command": exit st...  09-30 21:35:39
1-regenie-pl...  Stopped running "user-command": exit st...  09-30 23:08:16
1-regenie-pl...  Stopped running "user-command": exit st...  09-30 23:28:05
1-regenie-pl...

In [12]:
# Check the status of your job submissions

!dstat \
    --provider google-cls-v2 \
    --project terra-vpc-sc-4e1b6fe8 \
    --location us-central1 \
    --jobs '*' \
    --users 'jon126' \
    --status '*' \
       --full

- create-time: '2024-10-01 11:31:53.492403'
  dsub-version: v0-5-0
  end-time: ''
  envs:
    GOOGLE_PROJECT: terra-vpc-sc-4e1b6fe8
    chr: '15'
  events:
  - name: start
    start-time: 2024-10-01 11:32:04.225513+00:00
  - name: pulling-image
    start-time: 2024-10-01 11:32:58.581620+00:00
  - name: localizing-files
    start-time: 2024-10-01 11:46:57.435317+00:00
  - name: running-docker
    start-time: 2024-10-01 12:29:26.709590+00:00
  input-recursives: {}
  inputs:
    bedfile: gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr15.bed
    bimfile: gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr15.bim
    famfile: gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr15.fam
  internal-id: projects/282373700333/locations/us-central1/operations/1771003859732048182
  job-id: 1-regenie---jon126--241001-113153-36
  job

In [17]:
#Look at log file
!gsutil cp gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/logs/1-regenie-plinkprep/jon126/20241001/113152/1-regenie---jon126--241001-113153-36-task-None.log ./

Copying gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/logs/1-regenie-plinkprep/jon126/20241001/113152/1-regenie---jon126--241001-113153-36-task-None.log...
/ [1 files][266.9 KiB/266.9 KiB]                                                
Operation completed over 1 objects/266.9 KiB.                                    


In [13]:
#Cancel running jobs
# !ddel --provider google-cls-v2 --project terra-vpc-sc-4e1b6fe8 --jobs '*' --users 'jon126' --location us-central1

Delete running jobs:
  user:
    {'jon126'}

  job-id:
    ['*']

Found 0 tasks to delete.
0 jobs deleted


## REGENIE GWAS Step

### DSub Parameter Setting

This requires modification for each different phenotype you run.

In [3]:
## MODIFY FOR FULL DATA RUN 
JOB_NAME='REGENIE_ntprobnp'
PHENOTYPE='ntprobnp'

# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env JOB_NAME={JOB_NAME}
%env PHENOTYPE={PHENOTYPE}

env: JOB_NAME=REGENIE_ntprobnp


In [4]:
## Analysis Results Folder 
line_count_results_folder = os.path.join(
    os.getenv('WORKSPACE_BUCKET'),
    'dsub',
    'results',
    JOB_NAME,
    USER_NAME,
    datetime.now().strftime('%Y%m%d'))

line_count_results_folder

'gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/results/REGENIE_ntprobnp/jon126/20240927'

In [6]:
## Where the output files will go
output_files = os.path.join(line_count_results_folder, "results")
print(output_files)

OUTPUT_FILES = output_files

# Save this Python variable as an environment variable so that its easier to use within %%bash cells.
%env OUTPUT_FILES={OUTPUT_FILES}

gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/results/REGENIE_ntprobnp/jon126/20240927/results
env: OUTPUT_FILES=gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/results/REGENIE_ntprobnp/jon126/20240927/results


### REGENIE Bash Script

This details and writes out a .sh script in the local Jupyter disk and then uploads it to GCP Bucket in order for dsub to run it.

In [31]:
#This is the actual REGENIE bash script
filename2='2_regenie_aous_pp.sh'

script = '''
set -o pipefail 
set -o errexit

#This defines the actual bed_prefix, assuming localisation of the input bed/bim/fam files

echo "${bedfile}"
echo "${bimfile}"
echo "${famfile}"

bed_prefix="/mnt/data/input/gs/fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr"${chrom}" 

regenie \
    --step 1 \
    --bed "${bed_prefix}" \
    --phenoFile "${pheno_file}" \
    --phenoCol "${pheno}" \
    --covarFile "${cov_file}" \
    --catCovarList sex \
    --covarColList ht, wt, pc1, pc2, pc3, pc4, pc5, pc6, pc7, pc8, pc9, pc10, age_"${pheno}" \
    --bsize 1000 \
    --extract "${step1_snplist}" \
    --verbose \
    --"${trait}" \ 
    --apply-rint \
    --ref-first \
    --out "${pheno}"_step1_chr"${chrom}"

#regenie pt 2
regenie \
    --step 2 \
    --bed "${bed_prefix}" \
    --phenoFile "${pheno_file}" \
    --phenoCol "${pheno}" \
    --covarFile "${cov_file}" \
    --catCovarList sex \
    --covarColList ht, wt, pc1, pc2, pc3, pc4, pc5, pc6, pc7, pc8, pc9, pc10, age_"${pheno}" \
    --pred "${pheno}"_step1_chr"${chrom}"_pred.list \
    --bsize 1000 \
    --verbose \
    --"${trait}" \
    --apply-rint \
    --ref-first \
    --out "${pheno}"_step2_chr"${chrom}"

export regenie_results="${pheno}_step2_chr${chrom}.regenie"
echo "regenie_results: ${regenie_results}"
mv ${regenie_results} ${OUTPUT_PATH}
'''

with open(filename2,'w') as fp:
    fp.write(script)

In [32]:
#Upload to GCP Bucket
!gsutil cp ./2_regenie_aous_pp.sh {my_bucket}/dsub/scripts/

Copying file://./1_regenie_plinkprep.sh [Content-Type=text/x-sh]...
Copying file://./2_regenie_aous_pp.sh [Content-Type=text/x-sh]...               
CommandException: No URLs matched: -I                                           


In [25]:
#Check the files are there
!gsutil ls {my_bucket}/dsub/scripts/*.sh

gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/scripts/1_regenie_plinkprep.sh
gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/scripts/2_regenie_aous_pp.sh


### Dsub Submission Script for REGENIE

Note that all `--input` files have to be in double quotations 
whereas all `--env` environmental variables (for dsub) are NOT in quotations e.g `--env chrom=${chromo}` so if you need to use a Bash environmental variable for `--env` (in dsub) you remove the double quotes e.g `--env GOOGLE_PROJECT=${GOOGLE_PROJECT}`

Bash environmental variables are in `"${...}"` format 

In [19]:
!echo "${ARTIFACT_REGISTRY_DOCKER_REPO}" #Base location for public Docker images via Dockerhub

us-central1-docker.pkg.dev/all-of-us-rw-prod/aou-rw-gar-remote-repo-docker-prod


In [27]:
#This submits the job btw for each chromosome!
%%bash --out LINE_COUNT_JOB_ID

# Get a shorter username to leave more characters for the job name.
DSUB_USER_NAME="$(echo "${OWNER_EMAIL}" | cut -d@ -f1)"

# For AoU RWB projects network name is "network".
AOU_NETWORK=network
AOU_SUBNETWORK=subnetwork

MACHINE_TYPE="n2-standard-4"
BASH_SCRIPT="gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/dsub/scripts/2_regenie_aous_pp.sh" #From above command

# Python is 'right side limited' wherein the last value is not included
# To run the regression across all chromosomes, set lower to 1 and upper to 23
# To run across one chromosome, set lower to the chomosome-of-interest and upper to the following

LOWER=1
UPPER=23
for ((chromo=$LOWER;chromo<$UPPER;chromo+=1))
do
    dsub \
    --provider google-cls-v2 \
    --user-project "${GOOGLE_PROJECT}" \
    --project "${GOOGLE_PROJECT}" \
    --image "${ARTIFACT_REGISTRY_DOCKER_REPO}/skoyamamd/regenie_3.4.1:latest" \
    --network "${AOU_NETWORK}" \
    --subnetwork "${AOU_SUBNETWORK}" \
    --service-account "$(gcloud config get-value account)" \
    --user "${DSUB_USER_NAME}" \
    --regions us-central1 \
    --logging "${WORKSPACE_BUCKET}/dsub/logs/{job-name}/{user-id}/$(date +'%Y%m%d/%H%M%S')/{job-id}-{task-id}-{task-attempt}.log" \
    "$@" \
    --preemptible \
    --boot-disk-size 1000 \
    --machine-type ${MACHINE_TYPE} \
    --name "${JOB_NAME}" \
    --script "${BASH_SCRIPT}" \
    --env GOOGLE_PROJECT=${GOOGLE_PROJECT} \
    --input bedfile="gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr${chromo}.bed" \
    --input bimfile="gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr${chromo}.bim" \
    --input famfile="gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr${chromo}.fam" \
    --input pheno_file="gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/PP/data/pp_regenie_pheno.tsv" \
    --input cov_file="gs://fc-secure-7953e92c-a6a6-42df-9f19-86d553a9044f/PP/data/pp_regenie_covar.tsv" \
    --env chrom=${chromo} \
    --env pheno=${PHENOTYPE} \ 
    --env trait=qt \
    --output-recursive OUTPUT_PATH="${OUTPUT_FILES}/${chromo}"
done

Job properties:
  job-id: regenie-nt--jon126--240927-123947-19
  job-name: regenie-ntprobnp
  user-id: jon126
Provider internal-id (operation): projects/282373700333/locations/us-central1/operations/2520027973218601661
Launched job-id: regenie-nt--jon126--240927-123947-19
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-4e1b6fe8 --location us-central1 --jobs 'regenie-nt--jon126--240927-123947-19' --users 'jon126' --status '*'
To cancel the job, run:
  ddel --provider google-cls-v2 --project terra-vpc-sc-4e1b6fe8 --location us-central1 --jobs 'regenie-nt--jon126--240927-123947-19' --users 'jon126'
Job properties:
  job-id: regenie-nt--jon126--240927-123949-26
  job-name: regenie-ntprobnp
  user-id: jon126
Provider internal-id (operation): projects/282373700333/locations/us-central1/operations/16075055557443499835
Launched job-id: regenie-nt--jon126--240927-123949-26
To check the status, run:
  dstat --provider google-cls-v2 --project terra-vpc-sc-4e1b6fe

In [35]:
# Check the status of your job submissions

!dstat \
    --provider google-cls-v2 \
    --project terra-vpc-sc-4e1b6fe8 \
    --location us-central1 \
    --jobs '*' \
    --users 'jon126' \
    --status '*' \
   # --full

Job Name         Status                          Last Update
---------------  ------------------------------  --------------
regenie-ntpr...  Started running "user-command"  09-27 13:17:02
regenie-ntpr...  Started running "user-command"  09-27 13:15:38
regenie-ntpr...  Started running "localization"  09-27 12:56:17
regenie-ntpr...  Started running "localization"  09-27 12:57:12
regenie-ntpr...  Started running "localization"  09-27 12:55:47
regenie-ntpr...  Started running "localization"  09-27 12:56:08
regenie-ntpr...  Started running "localization"  09-27 12:55:28
regenie-ntpr...  Started running "localization"  09-27 12:54:41
regenie-ntpr...  Started running "localization"  09-27 12:55:50
regenie-ntpr...  Started running "localization"  09-27 12:55:30
regenie-ntpr...  Started running "localization"  09-27 12:55:49
regenie-ntpr...  Started running "localization"  09-27 12:55:35
regenie-ntpr...  Started running "localization"  09-27 12:54:30
regenie-ntpr...  Started running "localizat